# Cognitive Age Prediction with Structural and Functional MRI

## Structural MRI

In [80]:
# helper functions
import nibabel as nb
import numpy as np

def _vectorize_fs_surf(file_path):
    """
    Read surface information from a file and turn it into a vector.
    
    Parameters
    ----------
    file_path : str
        The path to a file with surface data.
    
    Returns
    -------
    vectorized_data : numpy.ndarray
        Extracted data.
    """
    img = nb.load(file_path)
    in_data = img.get_data().squeeze()

    return in_data


def get_area(subject_dir):
    """
    Read area information for the given subject and turn it into a vector.
    Data for left and right hemispheres are concatenated.
    
    Parameters
    ----------
    subject_dir : str
        The directory to files with surface data.
    
    Returns
    -------
    : numpy.ndarray
        Extracted data.
    """
    AREA_FILES = ('lh.area.mgh',  'rh.area.mgh')
    
    lh_data = _vectorize_fs_surf(join(subject_dir, AREA_FILES[0]))
    rh_data = _vectorize_fs_surf(join(subject_dir, AREA_FILES[1]))
    
    return np.concatenate((lh_data, rh_data), 0)


def get_thickness(subject_dir):
    """
    Read thickness information for the given subject and turn it into a vector.
    Data for left and right hemispheres are concatenated.
    
    Parameters
    ----------
    subject_dir : str
        The directory to files with surface data.
    
    Returns
    -------
    : numpy.ndarray
        Extracted data.
    """
    THICKNESS_FILES = ('rh.thickness.mgh', 'lh.thickness.mgh')
    
    lh_data = _vectorize_fs_surf(join(subject_dir, THICKNESS_FILES[0]))
    rh_data = _vectorize_fs_surf(join(subject_dir, THICKNESS_FILES[1]))
    
    return np.concatenate((lh_data, rh_data), 0)

In [83]:
import os
from os.path import isdir, join

import pandas as pd

# prepare freesurfer data
STRUCTURAL_DATA = '/home/okozyn/Projects/inria/camcan_analysis/data/structural'
VOLUME_FILE = 'aseg.csv'

subjects = tuple(d for d in os.listdir(STRUCTURAL_DATA) if isdir(join(STRUCTURAL_DATA, d)))

area_data = None
thickness_data = None
volume_data = None

area_failed = []
thickness_failes = []
volume_failed = []

for s in subjects:
    subject_dir = join(STRUCTURAL_DATA, s)
    
    try:
        t_area = get_area(subject_dir)
    except:
        area_failed.append(s)

    try:
        t_thickness = get_thickness(subject_dir)
    except:
        volume_failed.append(s)

    try:
        volume = pd.read_csv(join(subject_dir, VOLUME_FILE), index_col=0)
    except:
        volume_failed.append(s)

    if area_data is None:
        area_data = pd.DataFrame(index=subjects, columns=np.arange(start=0, stop=len(t_area)), dtype=float)
        area_data.loc[s] = t_area
    else:
        area_data.loc[s] = t_area
    
    if thickness_data is None:
        thickness_data = pd.DataFrame(index=subjects, columns=np.arange(start=0, stop=len(t_thickness)), dtype=float)
        thickness_data.loc[s] = t_thickness
    else:
        thickness_data.loc[s] = t_thickness

    if volume_data is None:
        volume_data = volume
    else:
        volume_data = pd.concat([volume_data, volume])

print('Failed to load area data for\n', area_failed)
print('Failed to load thickness data for\n', thickness_failed)
print('Failed to load volume data for\n', volume_failed)

area_data.to_csv(join(STRUCTURAL_DATA, 'area_data.csv'))
thickness_data.to_csv(join(STRUCTURAL_DATA, 'thickness_data.csv'))
volume_data.to_csv(join(STRUCTURAL_DATA, 'volume_data.csv'))

## Functional MRI

In [22]:
# load connectivity matrices
ATLASES = ['msdl', 'basc064', 'basc122', 'basc197']
# path for the different kind of connectivity matrices
CONNECTIVITY_KIND = ['correlation', 'partial correlation', 'tangent']

sel_atlas = ATLASES[1]
sel_connect = CONNECTIVITY_KIND[0]
sel_subject = 'sub-CC510355'

file_path = f'../../data/connectivity/{sel_subject}/{sel_atlas}/{sel_connect}/'\
            f'{sel_subject}_task-Rest_confounds.pkl'

In [23]:
with open(file_path, 'rb') as f:
    connect_matrix = joblib.load(f)

In [25]:
connect_matrix

array([0.80780958, 0.62181548, 0.69468642, ..., 0.58771813, 0.59074507,
       0.69387215])

scipy stats for Fichers transform
https://en.wikipedia.org/wiki/Fisher_transformation

This is just a tanh function

In [28]:
from itertools import product
meas = ('thickness', 'area')
hemi = ('lh', 'rh')

for h, m in product(hemi, meas):
    print(h, m)

lh thickness
lh area
rh thickness
rh area


In [1]:
from camcan import utils

In [35]:
import nibabel as nb
import numpy as np

img = nb.load(out_files['lh_thickness_file'])
in_data = img.get_data().squeeze()
# column vector
vectorized_data = in_data[:, np.newaxis]

In [36]:
t = np.genfromtxt(out_files['aseg_file'], skip_header=1, delimiter=',')[1:]

In [37]:
out_files.keys()

dict_keys(['lh_thickness_file', 'lh_area_file', 'rh_thickness_file', 'rh_area_file', 'aseg_file'])

In [38]:
t.shape

(66,)

In [33]:
vectorized_data.sort()
np.sum(vectorized_data > 0)

149600

In [39]:
img.get_data().shape

(163842, 1, 1)

In [ ]:
d = [d for d in os.listdir('./') if isdir(d) and 'CC' in d]  

In [2]:
from camcan.preprocessing.structural import get_cortex_data

In [4]:
subjects_dir = "/home/okozyn/Projects/inria/camcan_analysis/data/freesurfer"
subject = "CC620466"
out_dir = "/home/okozyn/Projects/inria/camcan_analysis/data/debug"

get_cortex_data(subjects_dir, subject, out_dir)

INFO:root:nsubjects = 1
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.8801
/home/okozyn/Projects/inria/camcan_analysis/notebook/mri
Log file is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mris_preproc.log
ven  1 mar 2019, 15.58.47, CET
setenv SUBJECTS_DIR /home/okozyn/Projects/inria/camcan_analysis/data/freesurfer
cd /home/okozyn/Projects/inria/camcan_analysis/notebook/mri
/home/okozyn/Programs/freesurfer/bin/mris_preproc --s CC620466 --target fsaverage4 --hemi lh --meas thickness --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mgh

Linux azzaro 4.15.0-45-generic #48-Ubuntu SMP Tue Jan 29 16:28:13 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
$Id: mris_preproc,v 1.76.2.1 2016/08/12 17:55:45 zkaufman Exp $
freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.8801
Src lh fsaverage4.sphere

ERROR:root:Non zero return code: 1. Bash: b'', cmd: mris_preproc --s CC620466 --target fsaverage4 --hemi rh --meas thickness --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.thickness.mgh
INFO:root:nsubjects = 1
INFO: turning on jacobican correction
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.9016
/home/okozyn/Projects/inria/camcan_analysis/notebook/mri
Log file is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mris_preproc.log
ven  1 mar 2019, 15.58.48, CET
setenv SUBJECTS_DIR /home/okozyn/Projects/inria/camcan_analysis/data/freesurfer
cd /home/okozyn/Projects/inria/camcan_analysis/notebook/mri
/home/okozyn/Programs/freesurfer/bin/mris_preproc --s CC620466 --target fsaverage4 --hemi rh --meas area --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mgh

Linux azzaro 4.15.0-45-generic #48-Ubuntu SMP Tue Jan 29 16:28:13 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
$Id: mr

{'lh_thickness_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mgh',
 'lh_area_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.area.mgh',
 'rh_thickness_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.thickness.mgh',
 'rh_area_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mgh'}